#  Notes

- I know nothing about Basketball so let's see how this goes
- The feature engineering seriously needs to be worked on
- Seeds do not change during the season
- Predict the probability of the lower TeamID winning
- Only rely on given data (just to make it easier)
- Possibly make a df for team statistics throughout the years (like if a team has been has been active since 1990, you do 1 row for that team per year). If you do this, make a separate df for women.
- Possibly make a df that's just for games where it shows who won.
- Somehow combine these into something a model can use to train on? This I need to think a lot about. If I have 3 different dataframes, with each giving me different kinds of data, how do I make a ML model that can train on those and predict the mashups of March Madness in the correct format?
- Because of the last bullet, try to not do any feature engineering or creating dataframes until you think about how a model can use it.
- After making dataframes, check dtypes

# Current game plan for features to attempt

### MTeams and WTeams
- **TeamID:** this is the unique team ID (does not repeat in this csv)
  - 4 digit number
- **TeamName:** short spelling of the team's name. this can be good just for a reference (not for prediction)
  - 16 character name
- **FirstD1Season:** the year that they became D1. these start at 1985
  - 4 digit number (1985-2025)
- **LastD1Season:** the last year they were D1.
  - 4 digit number (1985-2025)

### MNCAATourneySeeds and WNCAATourneySeeds
- **Season:** year of the season
  - 4 digit number (1985-2024)
- **Seed:** seed of the team
  - last 2 characters are the seed
- **TeamID:** team id of the season and seed
  - 4 digit number

### MNCAATourneySlots and WNCAATourneySlots
- **Season:** year of season
  - 4 digit number (1985-2024)
- **Slot:** slot that gives information on the seed from the winning or favored team
  - for play-ins, it is a 3 character string where the last 2 numbers are the winning seed. for regular tournaments, it is a 4 character string where the second character tells you the round, and the last character (or 2) tells you the expected seed of the favored team
- **StrongSeed:** seed that's strong
  - last 2 characters are the seed
- **WeakSeed:** weaker seed
  - last 2 characters are the seed

### MRegularSeasonDetailedResults and WRegularSeasonDetailedResults
- **Season:** year of the season
  - 4 digit number (2023-2025)
- **DayNum**: higher the number, the better the team is because they lasted longer
  - number
- **WTeamID:** winning team id
  - 4 digit number
- **WScore:** winning score
  - number
- **LTeamID:** losting team id
  - 4 digit number
- **LScore:** losing score
  - number
- **WLoc:** location of winning team
  - character (H, A, or N)
- **NumOT:** number of overtime periods
  - number
- **WFGM:** field goals made by winning team
  - number

- **OOPS! There are way more columns that are good, I just didn't see them before. Use them all...**

### MNCAATourneyDetailedResults and WNCAATourneyDetailedResults
- same as (M/W)RegularSeasonDetailedResults

### MSecondaryTourneyTeams and WSecondaryTournamentTeams
- **Season:** year
  - 4 digit number (1985-2024)
- **SecondaryTourney:** abbreviation of tournament
  - 3 characters
- **TeamID:** team that played in it
  - 4 digit number

### MSecondaryTourneyCompactResults and WSecondaryTourneyCompactResults
- all of this is the same as (M/W)RegularSeasonDetailedResults except there is no WFGM and instead it has the SecondaryTourney column and no team box scores

### Cities
- **CityID:** unique city id
  - number
- **State:** state abbreviation
  - 2 characters

### MGameCities and WGameCities
- **Season:** year
  - 4 digit number (2010-2025)
- **DayNum**: higher is better
  - number
- **WTeamID:** winning team
  - 4 digit number
- **LTeamID:** losing team
  - 4 digit number
- **CRType:** says whether the game is regular, ncaa, or secondary
  - characters
- **CityID:** unique city id
  - 4 digit number

### Other features to possibly make
- Win/lose ratio (for each team)
- Score of left team - score of right team difference (for that game)
- Calculate ELO rating per team

### Target
- I think a good target would be to make it so if each row is for 2 competing teams, show a 1 if the left team won, and a 0 if they lost. Then, the ML model will predict this and show a probability for it.
- I saw someone did it where instead of it being a 1 if they actually won, they did it so that the smaller TeamID is 1 and the other is 0. Not sure which route to go.


- Actually, maybe just make it into 1 df.

### Dataframe Structure
- **ID:** of team mashup (YYYY_T1ID_T2ID) from SampleSubmissionStage1
- **Year:** YYYY taken from SampleSubmissionStage1

- **LTeam:** taken from SampleSubmissionStage1
- **LTeamName:** taken from TeamName from (M/W)Teams (just to see team name, not for training)

- **RTeam:** taken from SampleSubmissionStage1
- **RTeamName:** taken from TeamName from (M/W)Teams (just to see team name, not for training)

- **D1Diff:** taken from subtracting the left LastD1Season - FirstD1Season from the right from (M/W)Teams
- **SeedDiff:** taken from MNCAATourneySeeds from subtracting the left seed from the right
- **FavoredSeed:** taken from MNCAATourneySlots where it's 1 if the left team is the favored seed (if it's all 1 then remove this)

- **RegELODiff:** which uses the data from (M/W)RegularSeasonDetailedResults to calculate the difference in ELO
- **TourneyELODiff:** which uses the data from (M/W)NCAATourneyDetailedResults to calculate the difference in ELO

- **Pred:** where 1 is if the left team won the game or 0 if they didn't.

# WHAT?????????????
- Please review all of the code and figure out exactly how the feature selection works
- why are you clipping predictions
- why use a regressor instead of a classifier
- why are missing values filled with -1
- is the standard scaler scaling any categorical columns??
- why are we fitting on X and then predicting with X? is this not the same as fitting with training data and then predicting with training data?

In [232]:
# Libraries
import pandas as pd # I wanted to use fireducks instead, but it was giving me issues
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import brier_score_loss, log_loss, mean_absolute_error
from sklearn.model_selection import cross_val_score

from xgboost import XGBRegressor

import glob
import os

In [233]:
# Getting all files
path = "./kaggle/input/march-machine-learning-mania-2025/**"
data = {p.split('/')[-1].split('.')[0].split('\\')[1] : pd.read_csv(p, encoding='latin-1') for p in glob.glob(path)}
# 遍历每一个文件，创建字典的键，把表格放在对应键后面
#data = {os.path.splitext(os.path.basename(p))[0]: pd.read_csv(p, encoding='latin-1') for p in glob.glob(path, recursive=True)} 
data

{'Cities':      CityID          City State
 0      4001       Abilene    TX
 1      4002         Akron    OH
 2      4003        Albany    NY
 3      4004   Albuquerque    NM
 4      4005     Allentown    PA
 ..      ...           ...   ...
 497    4520     Stateline    NV
 498    4521      Westwego    LA
 499    4522     Frankfort    KY
 500    4523      Rockford    IL
 501    4524  Collierville    TN
 
 [502 rows x 3 columns],
 'Conferences':     ConfAbbrev                                   Description
 0        a_sun                       Atlantic Sun Conference
 1        a_ten                        Atlantic 10 Conference
 2          aac                  American Athletic Conference
 3          acc                     Atlantic Coast Conference
 4          aec                       America East Conference
 5          asc                     American South Conference
 6          awc                      American West Conference
 7     big_east                           Big East Confe

In [234]:
# Loading data
"""
MTeams = data["MTeams"]
WTeams = data["WTeams"]

MNCAATourneySeeds = data["MNCAATourneySeeds"]
WNCAATourneySeeds = data["WNCAATourneySeeds"]

MNCAATourneySlots = data["MNCAATourneySlots"]
WNCAATourneySlots = data["WNCAATourneySlots"]

MRegularSeasonDetailedResults = data["MRegularSeasonDetailedResults"]
WRegularSeasonDetailedResults = data["WRegularSeasonDetailedResults"]

MNCAATourneyDetailedResults = data["MNCAATourneyDetailedResults"]
WNCAATourneyDetailedResults = data["WNCAATourneyDetailedResults"]

MSecondaryTourneyTeams = data["MSecondaryTourneyTeams"]
WSecondaryTourneyTeams = data["WSecondaryTourneyTeams"]

MSecondaryTourneyCompactResults = data["MSecondaryTourneyCompactResults"]
WSecondaryTourneyCompactResults = data["WSecondaryTourneyCompactResults"]

Cities = data["Cities"]

MGameCities = data["MGameCities"]
WGameCities = data["WGameCities"]
"""

# To be used for storing data, and later to take the IDs
df = data["SampleSubmissionStage2"]
df

,ID,Pred
0,2025_1101_1102,0.5
1,2025_1101_1103,0.5
2,2025_1101_1104,0.5
3,2025_1101_1105,0.5
4,2025_1101_1106,0.5
...,...,...
131402,2025_3477_3479,0.5
131403,2025_3477_3480,0.5
131404,2025_3478_3479,0.5
131405,2025_3478_3480,0.5


# Feature Engineering

In [235]:
# Creating year, left team, and right team columns
"""
df['Year'] = [int(yr[0:4]) for yr in df['ID']]
df['LTeam'] = [int(L[5:9]) for L in df['ID']]
df['RTeam'] = [int(R[10:14]) for R in df['ID']]
"""
df['RTeam'] = [int(R[10:14]) for R in df['ID']]
df['LTeam'] = [int(L[5:9]) for L in df['ID']]

df['LTeam']
df['ID'] # 从示例提交文件格式中获取左侧和右侧队伍的id


0         2025_1101_1102
1         2025_1101_1103
2         2025_1101_1104
3         2025_1101_1105
4         2025_1101_1106
               ...      
131402    2025_3477_3479
131403    2025_3477_3480
131404    2025_3478_3479
131405    2025_3478_3480
131406    2025_3479_3480
Name: ID, Length: 131407, dtype: object

In [236]:
# Lots of feature selecting and engineering
teams = pd.concat([data['MTeams'], data['WTeams']])
teams_spelling = pd.concat([data['MTeamSpellings'], data['WTeamSpellings']])
teams_spelling = teams_spelling.groupby(by='TeamID', as_index=False)['TeamNameSpelling'].count()
teams_spelling.columns = ['TeamID', 'TeamNameCount']
teams = pd.merge(teams, teams_spelling, how='left', on=['TeamID'])
del teams_spelling
teams.columns

Index(['TeamID', 'TeamName', 'FirstD1Season', 'LastD1Season', 'TeamNameCount'], dtype='object')

In [237]:
season_cresults = pd.concat([data['MRegularSeasonCompactResults'], data['WRegularSeasonCompactResults']])
season_dresults = pd.concat([data['MRegularSeasonDetailedResults'], data['WRegularSeasonDetailedResults']])
tourney_cresults = pd.concat([data['MNCAATourneyCompactResults'], data['WNCAATourneyCompactResults']])
tourney_dresults = pd.concat([data['MNCAATourneyDetailedResults'], data['WNCAATourneyDetailedResults']])
slots = pd.concat([data['MNCAATourneySlots'], data['WNCAATourneySlots']])
seeds = pd.concat([data['MNCAATourneySeeds'], data['WNCAATourneySeeds']])
gcities = pd.concat([data['MGameCities'], data['WGameCities']])
seasons = pd.concat([data['MSeasons'], data['WSeasons']])
#data['SampleSubmissionStage2']
tourney_cresults

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0
...,...,...,...,...,...,...,...,...
1645,2024,147,3163,80,3425,73,A,0
1646,2024,147,3234,94,3261,87,H,0
1647,2024,151,3234,71,3163,69,N,0
1648,2024,151,3376,78,3301,59,N,0


In [238]:
seeds = {'_'.join(map(str,[int(k1),k2])):int(v[1:3]) for k1, v, k2 in seeds[['Season', 'Seed', 'TeamID']].values}
cities = data['Cities']
sub = data['SampleSubmissionStage2']
del data
seeds.keys()
seeds['2007_1272'] # 哪一年 哪只队是几号种子。键：年和队，值：种子

2

In [239]:
season_cresults['ST'] = 'S'
season_dresults['ST'] = 'S'
tourney_cresults['ST'] = 'T'
tourney_dresults['ST'] = 'T'
games = pd.concat((season_dresults, tourney_dresults), axis=0, ignore_index=True)# 只有2003年开始才有detailed results，这里舍弃了compact results
games.reset_index(drop=True, inplace=True)
games['WLoc'] = games['WLoc'].map({'A': 1, 'H': 2, 'N': 3})
# 为常规赛和锦标赛比赛数据添加标记（S表示常规赛，T表示锦标赛）。
# 将常规赛详细结果和锦标赛详细结果合并为一个games数据集。
# 将比赛场地（WLoc）从字符串（A、H、N）转换为数字（1、2、3）。
games

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,ST
0,2003,10,1104,68,1328,62,3,0,27,58,...,16,22,10,22,8,18,9,2,20,S
1,2003,10,1272,70,1393,63,3,0,26,62,...,9,20,20,25,7,12,8,6,16,S
2,2003,11,1266,73,1437,61,3,0,24,58,...,14,23,31,22,9,12,2,5,23,S
3,2003,11,1296,56,1457,50,3,0,18,38,...,8,15,17,20,9,19,4,3,23,S
4,2003,11,1400,77,1208,71,3,0,30,61,...,17,27,21,15,12,10,7,1,14,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200645,2024,147,3163,80,3425,73,1,0,28,58,...,18,20,10,25,10,9,6,4,20,T
200646,2024,147,3234,94,3261,87,2,0,32,69,...,11,17,21,28,15,13,6,6,21,T
200647,2024,151,3234,71,3163,69,3,0,27,59,...,3,4,6,22,21,14,15,1,18,T
200648,2024,151,3376,78,3301,59,3,0,33,66,...,13,18,10,18,5,12,9,1,9,T


In [240]:
games['ID'] = games.apply(lambda r: '_'.join(map(str, [r['Season']]+sorted([r['WTeamID'],r['LTeamID']]))), axis=1)# 比赛id：年，1队，2队
games['IDTeams'] = games.apply(lambda r: '_'.join(map(str, sorted([r['WTeamID'],r['LTeamID']]))), axis=1)# 1队，2队
games['Team1'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[0], axis=1)# 1队
games['Team2'] = games.apply(lambda r: sorted([r['WTeamID'],r['LTeamID']])[1], axis=1)
games['IDTeam1'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)# 年 1队
games['IDTeam2'] = games.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)

games['Team1Seed'] = games['IDTeam1'].map(seeds).fillna(0)
games['Team2Seed'] = games['IDTeam2'].map(seeds).fillna(0)

games['ScoreDiff'] = games['WScore'] - games['LScore']
games['Pred'] = games.apply(lambda r: 1. if sorted([r['WTeamID'],r['LTeamID']])[0]==r['WTeamID'] else 0., axis=1) # 1队赢了没
games['ScoreDiffNorm'] = games.apply(lambda r: r['ScoreDiff'] * -1 if r['Pred'] == 0. else r['ScoreDiff'], axis=1)
games['SeedDiff'] = games['Team1Seed'] - games['Team2Seed']
games = games.fillna(-1)

In [241]:
c_score_col = ['NumOT', 'WFGM', 'WFGA', 'WFGM3', 'WFGA3', 'WFTM', 'WFTA', 'WOR', 'WDR', 'WAst', 'WTO', 'WStl',
 'WBlk', 'WPF', 'LFGM', 'LFGA', 'LFGM3', 'LFGA3', 'LFTM', 'LFTA', 'LOR', 'LDR', 'LAst', 'LTO', 'LStl',
 'LBlk', 'LPF'] # 选择和比赛得分相关的列
c_score_agg = ['sum', 'mean', 'median', 'max', 'min', 'std', 'skew', 'nunique']
gb = games.groupby(by=['IDTeams']).agg({k: c_score_agg for k in c_score_col}).reset_index()
# groupby 分组，同一支队的比赛会被分到一起
# agg 对于 c_score_col 中的每一列，分别计算每个分组（即每一对队伍）的这些聚合统计值。
# 聚合操作后，结果是一个 MultiIndex DataFrame。调用 reset_index() 是为了将其转换为普通的 DataFrame，方便后续处理。
gb.columns = [''.join(c) + '_c_score' for c in gb.columns]

games = games[games['ST']=='T']
gb.head()

,IDTeams_c_score,NumOTsum_c_score,NumOTmean_c_score,NumOTmedian_c_score,NumOTmax_c_score,NumOTmin_c_score,NumOTstd_c_score,NumOTskew_c_score,NumOTnunique_c_score,WFGMsum_c_score,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,1101_1102,0,0.0,0.0,0,0,NaN,NaN,1,23,...,NaN,1,14,14.0,14.0,14,14,NaN,NaN,1
1,1101_1115,0,0.0,0.0,0,0,NaN,NaN,1,24,...,NaN,1,19,19.0,19.0,19,19,NaN,NaN,1
2,1101_1116,0,0.0,0.0,0,0,0.0,NaN,1,53,...,NaN,2,40,20.0,20.0,21,19,1.414214,NaN,2
3,1101_1117,0,0.0,0.0,0,0,0.0,NaN,1,59,...,NaN,1,48,24.0,24.0,24,24,0.000000,NaN,1
4,1101_1122,0,0.0,0.0,0,0,NaN,NaN,1,29,...,NaN,1,23,23.0,23.0,23,23,NaN,NaN,1


In [242]:
sub['WLoc'] = 3
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['ID'].map(lambda x: x.split('_')[0])
sub['Season'] = sub['Season'].astype(int)
sub['Team1'] = sub['ID'].map(lambda x: x.split('_')[1])
sub['Team2'] = sub['ID'].map(lambda x: x.split('_')[2])
sub['IDTeams'] = sub.apply(lambda r: '_'.join(map(str, [r['Team1'], r['Team2']])), axis=1)
sub['IDTeam1'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team1']])), axis=1)
sub['IDTeam2'] = sub.apply(lambda r: '_'.join(map(str, [r['Season'], r['Team2']])), axis=1)
sub['Team1Seed'] = sub['IDTeam1'].map(seeds).fillna(0)
sub['Team2Seed'] = sub['IDTeam2'].map(seeds).fillna(0)
sub['SeedDiff'] = sub['Team1Seed'] - sub['Team2Seed'] # 提取各种信息和添加种子特征
sub = sub.fillna(-1)

games = pd.merge(games, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
sub = pd.merge(sub, gb, how='left', left_on='IDTeams', right_on='IDTeams_c_score')
# 将比赛数据（games）与之前生成的统计特征数据（gb）通过IDTeams进行左连接。
# 将提交数据（sub）与统计特征数据（gb）通过IDTeams进行左连接。

col = [c for c in games.columns if c not in ['ID', 'DayNum', 'ST', 'Team1', 'Team2', 'IDTeams', 'IDTeam1', 'IDTeam2',
                                             'WTeamID', 'WScore', 'LTeamID', 'LScore', 'NumOT', 'Pred', 'ScoreDiff', 'ScoreDiffNorm',
                                             'WLoc'] + c_score_col]

In [243]:
games.head()
# #games.columns
# games['ID']
# games['IDTeams']
# games['Team1']

# games['IDTeam1']

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2003,134,1421,92,1411,84,3,1,32,69,...,NaN,1,22,22.0,22.0,22,22,NaN,NaN,1
1,2003,136,1112,80,1436,51,3,0,31,66,...,NaN,1,15,15.0,15.0,15,15,NaN,NaN,1
2,2003,136,1113,84,1272,71,3,0,31,59,...,NaN,1,18,18.0,18.0,18,18,NaN,NaN,1
3,2003,136,1141,79,1166,73,3,0,29,53,...,NaN,2,35,17.5,17.5,21,14,4.949747,NaN,2
4,2003,136,1143,76,1301,74,3,1,27,64,...,-0.422521,4,66,16.5,17.0,19,13,2.645751,-0.863919,4


In [244]:
sub.columns

Index(['ID', 'Pred', 'RTeam', 'LTeam', 'WLoc', 'Season', 'Team1', 'Team2',
       'IDTeams', 'IDTeam1',
       ...
       'LBlkskew_c_score', 'LBlknunique_c_score', 'LPFsum_c_score',
       'LPFmean_c_score', 'LPFmedian_c_score', 'LPFmax_c_score',
       'LPFmin_c_score', 'LPFstd_c_score', 'LPFskew_c_score',
       'LPFnunique_c_score'],
      dtype='object', length=231)

# Model

In [245]:
# Selecting training data
X = games[col].fillna(-1)
sub_X = sub[col].fillna(-1)

In [246]:
X

,Season,Team1Seed,Team2Seed,SeedDiff,IDTeams_c_score,NumOTsum_c_score,NumOTmean_c_score,NumOTmedian_c_score,NumOTmax_c_score,NumOTmin_c_score,...,LBlkskew_c_score,LBlknunique_c_score,LPFsum_c_score,LPFmean_c_score,LPFmedian_c_score,LPFmax_c_score,LPFmin_c_score,LPFstd_c_score,LPFskew_c_score,LPFnunique_c_score
0,2003,16.0,16.0,0.0,1411_1421,1,1.00,1.0,1,1,...,-1.000000,1,22,22.000000,22.0,22,22,-1.000000,-1.000000,1
1,2003,1.0,16.0,-15.0,1112_1436,0,0.00,0.0,0,0,...,-1.000000,1,15,15.000000,15.0,15,15,-1.000000,-1.000000,1
2,2003,10.0,7.0,3.0,1113_1272,0,0.00,0.0,0,0,...,-1.000000,1,18,18.000000,18.0,18,18,-1.000000,-1.000000,1
3,2003,11.0,6.0,5.0,1141_1166,0,0.00,0.0,0,0,...,-1.000000,2,35,17.500000,17.5,21,14,4.949747,-1.000000,2
4,2003,8.0,9.0,-1.0,1143_1301,1,0.25,0.0,1,0,...,-0.422521,4,66,16.500000,17.0,19,13,2.645751,-0.863919,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2271,2024,3.0,1.0,2.0,3163_3425,0,0.00,0.0,0,0,...,-1.000000,2,32,16.000000,16.0,20,12,5.656854,-1.000000,2
2272,2024,1.0,3.0,-2.0,3234_3261,0,0.00,0.0,0,0,...,-1.000000,2,40,20.000000,20.0,21,19,1.414214,-1.000000,2
2273,2024,3.0,1.0,2.0,3163_3234,0,0.00,0.0,0,0,...,1.732051,2,35,11.666667,10.0,18,7,5.686241,1.205659,3
2274,2024,3.0,1.0,2.0,3301_3376,0,0.00,0.0,0,0,...,0.190198,7,104,14.857143,15.0,22,8,5.014265,-0.159771,6


In [247]:
# GridSearchCV

# Do some grid search here for the parameters
# After doing the grid search, save the best parameters and create the param_grid in this cell

In [248]:
# XGB parameters
param_grid = {
    'n_estimators': 5000,
    'learning_rate': 0.03,
    'max_depth': 6
}

# Pipeline
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
])

# Fitting pipeline
pipeline.fit(X, games['Pred'])

# Predicting games and submissions
pred = pipeline.predict(X).clip(0.001, 0.999)
sub_pred = pipeline.predict(sub_X).clip(0.001, 0.999)

# Cross validation (for the MSE)
cv_scores = cross_val_score(pipeline, X, games['Pred'], cv=5, scoring="neg_mean_squared_error")

# Cross val

In [258]:
from sklearn.model_selection import train_test_split, KFold

# 划分训练集和验证集
X_train, X_val, y_train, y_val = train_test_split(X, games['Pred'], test_size=0.2, random_state=42)
#X_train
print(len(X_train),len(X_val), len(y_train), len(y_val), len(X), len(games['Pred']))

1820 456 1820 456 2276 2276


In [259]:
# 定义 XGB 参数
param_grid = {
    'n_estimators': 5000,
    'learning_rate': 0.03,
    'max_depth': 6
}

# 定义 Pipeline
pipeline2 = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler()),
    ('xgb', XGBRegressor(**param_grid, device="gpu", random_state=42))
])

pipeline2.fit(X_train, y_train)

pred2 = pipeline2.predict(X_val).clip(0.001, 0.999)
cv_scores2 = cross_val_score(pipeline2, X_val, y_val, cv=5, scoring="neg_mean_squared_error")



In [265]:
print(len(X_val), len(pred2))
cv_scores2 = cross_val_score(pipeline2, X, games['Pred'], cv=5, scoring="neg_mean_squared_error")


456 456


In [267]:
# # 使用 KFold 进行交叉验证
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # 对训练集进行交叉验证
# cv_scores = cross_val_score(pipeline, X_val, y_val, cv=kf, scoring='neg_mean_squared_error')

# 输出交叉验证结果
# print(f"交叉验证的负均方误差 (MSE): {cv_scores}")
# print(f"交叉验证的均方误差平均值: {-cv_scores.mean()}")

# # 如果需要，也可以对整个数据集进行交叉验证
# full_cv_scores = cross_val_score(pipeline, X, games['Pred'], cv=5, scoring='neg_mean_squared_error')
# print(f"全数据集交叉验证的负均方误差 (MSE): {full_cv_scores}")
# print(f"全数据集交叉验证的均方误差平均值: {-full_cv_scores.mean()}")

# Results
print(f'Log Loss: {log_loss(games["Pred"], pred):.5f}')
print(f'Mean Absolute Error: {mean_absolute_error(games["Pred"], pred):.5f}')
print(f'Brier Score: {brier_score_loss(games["Pred"], pred):.5f}')
print(f'Cross-validated MSE: {-cv_scores.mean():.5f}')
# Results
print(f'Log Loss: {log_loss(y_val, pred2):.5f}')
print(f'Mean Absolute Error: {mean_absolute_error(y_val, pred2):.5f}')
print(f'Brier Score: {brier_score_loss(y_val, pred2):.5f}')
print(f'Cross-validated MSE: {-cv_scores2.mean():.5f}')

Log Loss: 0.00163
Mean Absolute Error: 0.00163
Brier Score: 0.00001
Cross-validated MSE: 0.20514
Log Loss: 0.72878
Mean Absolute Error: 0.37782
Brier Score: 0.21727
Cross-validated MSE: 0.20514


In [263]:
print(len(games['Pred']))
print(len(pred))
print(len(y_val))
print(len(pred2))
type(X_train)

2276
2276
456
456


pandas.core.frame.DataFrame

# Results

In [187]:
# Results
print(f'Log Loss: {log_loss(games["Pred"], pred):.5f}')
print(f'Mean Absolute Error: {mean_absolute_error(games["Pred"], pred):.5f}')
print(f'Brier Score: {brier_score_loss(games["Pred"], pred):.5f}')
print(f'Cross-validated MSE: {-cv_scores.mean():.5f}')

Log Loss: 0.13169
Mean Absolute Error: 0.07534
Brier Score: 0.04177
Cross-validated MSE: 0.20040


# Submission

In [191]:
# Creating submission dataframe

sub_pred = pipeline.predict(sub_X).clip(0.001, 0.999)

submission_df = pd.DataFrame({
    'ID': df['ID'],
    'Pred': sub_pred
})

# Shape and head/tail of submission
print(f"{submission_df.shape} \n")
print(f"{submission_df.head().to_string(index=False)} \n")
print(submission_df.tail().to_string(index=False))

# Saving to csv
submission_df.to_csv('submission.csv', index=False)
print("\nSubmission file saved! Good luck!!! :)")

(131407, 2) 

            ID     Pred
2025_1101_1102 0.496842
2025_1101_1103 0.472241
2025_1101_1104 0.472241
2025_1101_1105 0.472241
2025_1101_1106 0.472241 

            ID     Pred
2025_3477_3479 0.472241
2025_3477_3480 0.472241
2025_3478_3479 0.591060
2025_3478_3480 0.472241
2025_3479_3480 0.472241

Submission file saved! Good luck!!! :)


# What I Learned

Write some things that you learned, struggled with, etc later.